In [1]:
!pip3 install -U paddlepaddle-gpu==2.3.2
!pip3 install -U paddlenlp==2.3.4
!pip3 install trustai==0.1.5

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.0/394.0 MB 975.4 kB/s eta 0:00:0000:0100:04
  Attempting uninstall: paddlepaddle-gpu
    Found existing installation: paddlepaddle-gpu 2.3.2.post112
    Uninstalling paddlepaddle-gpu-2.3.2.post112:
      Successfully uninstalled paddlepaddle-gpu-2.3.2.post112

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.9/441.9 kB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 768.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 558.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━

In [1]:
import re
import os
import sys
from tqdm import tqdm
import numpy as np
import copy
import json
import paddle
import paddlenlp
from paddlenlp.embeddings import TokenEmbedding
from paddlenlp.data import JiebaTokenizer
from paddlenlp.transformers import ErnieForQuestionAnswering, ErnieTokenizer
from utils import *
from mrc_utils import *
paddle.set_device("gpu")

# Select pre-trained model
MODEL_NAME = "ernie-3.0-xbase-zh" # choose from ["ernie-1.0", "ernie-1.0-base-zh", "ernie-1.0-large-zh-cw", "ernie-2.0-base-zh", "ernie-2.0-large-zh", "ernie-3.0-xbase-zh", "ernie-3.0-base-zh", "ernie-3.0-medium-zh", "ernie-3.0-mini-zh", "ernie-3.0-micro-zh", "ernie-3.0-nano-zh"]
# Select dataset for model training
DATASET_NAME = 'dureader_robust'
# Set the path to save the trained model
MODEL_SAVE_PATH = f'save_model/{DATASET_NAME}-{MODEL_NAME}'
# Set the rationale length ratio which determines the length of the extracted rationales.
RATIONALE_RATIO = 0.096 # 0.096 for Chinese dataset, 0.102 for English dataset

# Init model and tokenizer
model = ErnieForQuestionAnswering.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

# Correct MODEL_PATH and DATA_PATH before executing
MODEL_PATH = MODEL_SAVE_PATH + '/model_state.pdparams'

# Load the trained parameters
state_dict = paddle.load(MODEL_PATH)
model.set_dict(state_dict)

[2022-10-27 23:55:19,865] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-3.0-xbase-zh/ernie_3.0_xbase_zh.pdparams
W1027 23:55:19.869297  3198 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.2
W1027 23:55:19.872598  3198 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2022-10-27 23:55:24,946] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-3.0-xbase-zh/ernie_3.0_xbase_zh_vocab.txt
[2022-10-27 23:55:24,971] [    INFO] - tokenizer config file saved in /home/aistudio/.paddlenlp/models/ernie-3.0-xbase-zh/tokenizer_config.json
[2022-10-27 23:55:24,973] [    INFO] - Special tokens file saved in /home/aistudio/.paddlenlp/models/ernie-3.0-xbase-zh/special_tokens_map.json


### UTILS--WRITED--BY--MYSELF

In [2]:
WORK_ROOT = 'work/version1.2/'
DATA_PATH = WORK_ROOT + 'data/'
RESULT_PATH = WORK_ROOT + 'data/result/'
if not os.path.exists(WORK_ROOT):
    os.makedirs(WORK_ROOT)
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)

# 文本过滤后答案的丢失率
def answers_lost_rate(dataset):
    count = 0
    for example in dataset:
        for answer in example['answers']:
            if example['context'].count(answer):
                count += 1
                break
    rate = (len(dataset)-count) / len(dataset)
    return rate

# 文本过滤后的文本删除率
def rm_redundency_rate(original, preprocessed):
    original_count = 0
    preprocessed_count = 0
    for example in original:
        original_count += len(example['context'])
    for example in preprocessed:
        preprocessed_count += len(example['context'])

    rate = (original_count-preprocessed_count) / original_count
    return rate

# 自定义的文本过滤器
def further_select(dataset, data_type, threshold, rm_inte=True):
    # context，question抽取出来存储到documents和queries(List)
    documents = []
    queries = []
    for data in dataset:
        documents.append([data['context']])
        queries.append([data['question']])
    assert len(documents) == len(queries)

    # 对document中的每一个document进行标点符号的切分，将其分成一句一句的形式，以句号级别为分割，剔除疑问句
    max_sentence_num = -1
    min_sentence_num = 1000
    avg_sentence_num = 0
    documents_split = [re.split(r'[。！!|]', document[0].strip()) for document in documents]
    
    for document in documents_split:
        if len(document) > max_sentence_num:
            max_sentence_num = len(document)
        if len(document) < min_sentence_num:
            min_sentence_num = len(document)
        avg_sentence_num += len(document)
    avg_sentence_num /= len(dataset)
    
    print('该数据集中最长文档包含句数：', max_sentence_num, '\n该数据集中最短文档包含句数：', min_sentence_num, '\n该数据集中文档平均包含句数：', avg_sentence_num)

    if rm_inte:
        interrogative_words = set(['谁','何','么','哪','几时','几个','几十','几百','几千','几万','几亿','多少','怎','吗','呢','难道','岂','究竟','?','？','行不行','好不好','能不能','要不要'])
    else:
        interrogative_words = set([])

    documents_split_rm_inte = []
    for document in documents_split:
        d = []
        for sentences in document:
            '''
            if sentences != "":
                se = []
                sentence = re.split(r'[,，:：;； ]', sentences.strip())
                for s in sentence:
                    if s != "" and len(set(s)&interrogative_words) == 0:
                        se.append(s)
                if se != []:
                    d.append([",".join(se)])
            '''
            if sentences != "" and all([word not in sentences for word in interrogative_words]):
                d.append([sentences])
        if d != []:
            documents_split_rm_inte.append(d)
        else:
            documents_split_rm_inte.append([[sentence] for sentence in document if sentence != ""])

    # 导入词向量及分词模型
    token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")
    tokenizer = JiebaTokenizer(vocab=token_embedding.vocab)
    embedding = token_embedding.search("中国")

    # 对documents实现从自然语言 -> vector操作
    pbar = tqdm(total=len(documents_split_rm_inte), ncols=100, desc='已操作文档数目：')
    documents_vector = []

    for document in documents_split_rm_inte:  # 针对每一文档一一操作
        pbar.update(1)
        document_vector = []
        for sentence in document:  # 针对文档的每一句进行一一操作
            words = tokenizer.cut(sentence[0])
            word_embeds = np.zeros(embedding.shape)
            for word in words:
                word_embed = token_embedding.search(word)  # 句子分词
                word_embeds = np.append(word_embeds, word_embed, axis=0)
            word_embeds = np.delete(word_embeds, 0, axis=0)
            sentence_embed = np.mean(word_embeds, axis=0).tolist()  # 均值化词向量用于表示句向量
            assert len(sentence_embed) == 300
            document_vector.append(sentence_embed)
        if len(document_vector) < max_sentence_num:  # 填充文档长度一边快速计算
            need_padding_line_nums = max_sentence_num - len(document_vector)
            paddings = [[0]*len(sentence_embed) for i in range(need_padding_line_nums)]
            document_vector.extend(paddings)
        assert len(document_vector) == max_sentence_num
        documents_vector.append(document_vector)

    pbar.close()

    documents_tensor = paddle.to_tensor(documents_vector)

    # 实现queries从自然语言-> vector操作
    pbar = tqdm(total=len(documents_split_rm_inte), ncols=100, desc='已操作查询数目：')
    queries_vector = []

    for query in queries:
        pbar.update(1)
        words = tokenizer.cut(query[0])
        word_embeds = np.zeros(embedding.shape)
        for word in words:
            word_embed = token_embedding.search(word)
            word_embeds = np.append(word_embeds, word_embed, axis=0)
        word_embeds = np.delete(word_embeds, 0, axis=0)
        sentence_embed = np.mean(word_embeds, axis=0).tolist()  # 均值化词向量用于表示句向量
        assert len(sentence_embed) == 300
        queries_vector.append([sentence_embed])

    pbar.close()

    queries_tensor = paddle.to_tensor(queries_vector)

    # 计算分数矩阵并获取相似度的高的句子索引
    score_matrix = documents_tensor * queries_tensor
    score = score_matrix.sum(axis=-1)
    sign = -paddle.cast(score<0, dtype='float')
    sign[sign==0] = 1
    # print(sign[534])
    score *= sign

    # print(score[:2])
    index_min_to_max = paddle.argsort(score)
    index_max_to_min = [i[::-1] for i in index_min_to_max]
    # print(index_max_to_min[:2])

    max_value = score.max()
    score[score==0.0] = max_value + 1.0
    min_score = score.min(axis=1).unsqueeze(1)
    score[score==max_value+1.0] = 0.0
    # print(min_score[:2])
    
    score -= min_score * 4/5
    score[score<0] = 0.0
    sum_score = score.sum(axis=1).unsqueeze(1)
    # print(sum_score[:2])
    # print(score[:2])

    # 根据score选取topK个关键句(topK按占比进行选择)
    documents_split_topK = []
    for idx, document in enumerate(documents_split_rm_inte):
        # print(idx, document)
        index_sorted = index_max_to_min[idx][:len(score[idx].nonzero())]
        if len(index_sorted) <= 1:
            documents_split_topK.append(document)
            continue
        index_original = []
        ratio = 0.0
        d = []
        # print(index_sorted)
        for index in index_sorted:
            ratio += score[idx][index] / sum_score[idx]
            # print(ratio)
            index_original.append(index)
            if ratio >= threshold:
                break
        # print(index_original)
        index_original = sorted(index_original)
        # print(index_original)
        # print('**** ****')
        # print(index_original)
        for index in index_original:
            d.append(document[index])
        documents_split_topK.append(d)
    

    # 将关键句子按原序重组
    documents_split_topK_merge = [[sentence[0] for sentence in document] for document in documents_split_topK]
    documents_split_topK_merge = ["。".join(document)+'。' for document in documents_split_topK_merge]

    # 利用关键句构造一个新的json训练集
    # 由于contex被改变，answer的position也应当被改变（针对训练集）
    dataset_new = copy.deepcopy(dataset)
    assert dataset_new != dataset
    for i in range(len(dataset_new)):
        dataset_new[i]['context'] = documents_split_topK_merge[i].strip()
        if data_type == 'train':
            dataset_new[i]['answer_starts'][0] = documents_split_topK_merge[i].find(dataset_new[i]['answers'][0])
        if data_type == 'test':
            dataset_new[i]['sent_token'] = [w for w in dataset_new[i]['context']]

    return dataset_new


THRESHOLD = 0.8
def context_filter(dataset, filter_type, data_type, file_name):
    assert filter_type == 0 or (filter_type != 0 and file_name)
    dataset_filter = copy.deepcopy(dataset)
    preprocess = ['None', 'select', 'further_select', 'add_further_select']
    if preprocess[filter_type] == 'None':
        pass
    elif preprocess[filter_type] == 'select':
        with open(DATA_PATH + file_name) as f:
            dataset_select = json.load(f)
        for idx, example in enumerate(dataset_select['data'][0]['paragraphs']):
            dataset_filter[idx]['context'] = example['context']
            dataset_filter[idx]['sent_token'] = [word for word in example['context']]
    elif preprocess[filter_type] == 'further_select':
        with open(DATA_PATH + file_name) as f:
            dataset_select = json.load(f)
        for idx, example in enumerate(dataset_select['data'][0]['paragraphs']):
            dataset_filter[idx]['context'] = example['context']
        threshold = float(input('请输入further select句子分数之和的阈值'))
        THRESHOLD = threshold
        dataset_filter = further_select(dataset_filter, data_type, threshold)
    elif preprocess[filter_type] == 'add_further_select':
        data_file = DATA_PATH + file_name
        dataset_further_select = DuReader().read(data_file)
        for idx, example in enumerate(dataset_further_select):
            if dataset_filter[idx]['sent_token'][-1] not in '。？！?!':
                dataset_filter[idx]['context'] += '。' + example['context']
            else:
                dataset_filter[idx]['context'] += example['context']
            dataset_filter[idx]['context'] = dataset_filter[idx]['context']
            dataset_filter[idx]['sent_token'] = [word for word in dataset_filter[idx]['context']]
    
    if 'test' not in file_name:
        print('answer loss rate:', answers_lost_rate(dataset_filter))
    print('remove redundency rate:', rm_redundency_rate(dataset, dataset_filter))

    return dataset_filter


# 保存测试数据
def save_test_data(dataset, file_path):
    with open(file_path, 'w') as f:
        for example in dataset:
            s = json.dumps(example, ensure_ascii=False)
            f.writelines(s+'\n')

# 载入测试数据
def load_test_data(data_file):
    if os.path.exists(data_file):
        data_ds = DuReader().read(data_file)
        data = load_data(data_file)
    else:
        data_path = DATA_PATH + 'test.txt'
        data_ds = DuReader().read(data_path)
        if data_file.split('/')[-1].count('test_select'):
            data_ds = context_filter(data_ds, 1, 'test', 'test_select.json')
        elif data_file.split('/')[-1].count('test_further_select'):
            data_ds = context_filter(data_ds, 2, 'test', 'test_select.json')
        elif data_file.split('/')[-1].count('test_add_further_select'):
            data_ds = context_filter(data_ds, 3, 'test', 'test_further_select.txt')
        
        save_test_data(data_ds, data_file)
        data = load_data(data_file)
    
    print("Num of data:", len(data))

    return data_ds, data



def cal_sent_pos(dataset):
    context_split = []
    context_sent_pos = []
    for example in dataset:
        context_split.append([s for s in re.split(r'[。，；！？,;!?]', example['context'].strip()) if s != ""])
        sent_num = len(context_split[-1])
        sent_pos = [0]
        end_pos = 0
        for word in example['context']:  # 英文单词有拆分
            if word not in '。，；！？,;!?':
                end_pos += 1
            else:
                end_pos += 1  # 包含标点符号
                sent_pos.append(end_pos)
        if len(sent_pos) == sent_num:  # 确保包含头尾
            sent_pos.append(end_pos+1)
        context_sent_pos.append(sent_pos)
    
    print(dataset[0], '\n', context_sent_pos[0])
    
    return context_sent_pos



#### 2）数据预处理

a) 输入格式化：将输入的两个文本组织成模型预测所需格式，如对于Ernie3.0-base模型，其输入形式为[CLS]question[SEP]context[SEP]

b) 分词位置索引：计算每个分词结果对应的原文位置索引，这里的分词包括模型分词和标准分词

In [3]:
# DATA_FILE = DATA_PATH + 'test.txt'  # version 1.0
# DATA_FILE = DATA_PATH + 'test_select.txt'  # version 1.1
DATA_FILE = DATA_PATH + 'test_further_select.txt'  # version 1.2
# DATA_FILE = DATA_PATH + 'test_add_further_select.txt'  # version 1.3

data_ds, data = load_test_data(DATA_FILE)
context_sent_pos = cal_sent_pos(data_ds)

Num of data: 1855
{'id': 1, 'context': '地瓜不是红薯。红薯是粗粮，也叫番薯山芋。', 'question': '地瓜和红薯一样吗', 'sent_token': ['地', '瓜', '不', '是', '红', '薯', '。', '红', '薯', '是', '粗', '粮', '，', '也', '叫', '番', '薯', '山', '芋', '。']} 
 [0, 7, 13, 20]


In [10]:
from functools import partial
from mrc_utils import *
# Hyperparameters
batch_size = 12
max_seq_length = 512
epochs = 3  #3
warmup_proportion = 0.1
weight_decay = 0.01
doc_stride = 512

data_ds_new = copy.deepcopy(data_ds)

# Prepare dataloader
test_trans_func = partial(prepare_validation_features, 
                            max_seq_length=max_seq_length, 
                            doc_stride=doc_stride,
                            tokenizer=tokenizer)
print(len(data_ds_new))
                            
data_ds_new.map(test_trans_func, batched=True, num_workers=4)
test_batch_sampler = paddle.io.DistributedBatchSampler(
        data_ds_new, batch_size=batch_size, shuffle=False)

print(len(data_ds_new))

test_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
}): fn(samples)
test_data_loader = paddle.io.DataLoader(
    dataset=data_ds_new,
    batch_sampler=test_batch_sampler,
    collate_fn=test_batchify_fn,
    return_list=True)

# Get offset maps which will be used for score alignment
contexts, standard_split, ori_offset_maps, standard_split_offset_maps = pre_process(data, data_ds_new, tokenizer)

1855
1855


#### 3）重要度分数获取
我们提供attention和IG两种解释方法，可根据实际实验结果选取最有效的一种方法。

##### a） Attention-based Interpreter

In [11]:
from trustai.interpretation.token_level import AttentionInterpreter
from utils import create_dataloader_from_scratch
import paddle
# Hyperparameters
BATCH_SIZE = 64

# Init an attention interpreter and get the importance scores
att = AttentionInterpreter(model, device="gpu", predict_fn=attention_predict_fn)

# Use attention interpreter to get the importance scores for all data
interp_results = None
for batch in test_data_loader:
    if interp_results:
        interp_results += att(batch)
    else:
        interp_results = att(batch)

# Trim the output to get scores only for context
interp_results = trim_output(interp_results, data_ds_new, tokenizer)

# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = att.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

##### b) erasable-based Interpreter

In [12]:
import jieba.posseg as pseg
import pickle as pkl
import math


batch_size = 1
test_batch_sampler = paddle.io.DistributedBatchSampler(
        data_ds_new, batch_size=batch_size, shuffle=False)

test_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
}): fn(samples)
test_data_loader = paddle.io.DataLoader(
    dataset=data_ds_new,
    batch_sampler=test_batch_sampler,
    collate_fn=test_batchify_fn,
    return_list=True)

def MSE(y,t):
    #形参t代表训练数据（监督数据）（真实）
    #y代表预测数据
    return 0.5*np.sum((y-t)**2)

# 避免单词训练耗时，训练完保存
def save_word_loss(word_loss, file_path):
    all_loss = {}
    idx = 0
    for loss in word_loss:
        d = {}
        for l in loss:
            key, value = l
            d[key] = value
        all_loss[idx] = d
        idx += 1
    with open(file_path, 'wb') as f:
        pkl.dump(all_loss, f)

WORD_LOSS_FILE = DATA_PATH + 'word_loss.pkl'
word_loss = []
if os.path.exists(WORD_LOSS_FILE):
    with open(WORD_LOSS_FILE, 'rb') as f:
        word_loss_dict = pkl.load(f)
    for key in word_loss_dict:
        term = zip(word_loss_dict[key].keys(), word_loss_dict[key].values())
        loss = []
        for t in term:
            loss.append(t)
        word_loss.append(loss)
else:
    pbar = tqdm(total=len(data)/batch_size, ncols=100, desc='已操作文档数目：')

    for step, batch in enumerate(test_data_loader, start=0):
        pbar.update(1)
        data_id=data_ds[step]['id']
        results=align_res[step]
        topk = math.ceil(len(data[data_id]['sent_token'])*RATIONALE_RATIO)
        test_context=data[data_id]
        context=test_context['context']
        seg_list =pseg.cut(context)
        ner_list=[]
        for w in seg_list:
            ner_list.append(w)
        # print(ner_list)
        # print(dev_question[step])
        # global_step += 1
        input_ids, segment_ids= batch
        real_logits = model(input_ids=input_ids, token_type_ids=segment_ids)
        start_logits,end_logits=real_logits
        # print(start_logits.numpy())
        # print('\n',end_logits)
        input_ids=input_ids.numpy()
        start=0
        mse_losses={}
        mse_losses_flag={}
        for w in ner_list:
            word=w.word
            if word in[',','.','，','。','-','+','?','!']:
                continue
            flag=w.flag
            change_input_ids=input_ids[0]
            change_input_ids[start:start+len(word)]=0
            new_input_ids=[]
            new_input_ids.append(change_input_ids)
            new_input_ids=paddle.to_tensor(new_input_ids)
            change_logits=model(input_ids=new_input_ids, token_type_ids=segment_ids)
            new_start_logits,new_end_logits=change_logits
            mse_loss_start=MSE(start_logits.numpy()[0],new_start_logits.numpy()[0])
            mse_loss_end=MSE(end_logits.numpy()[0],new_end_logits.numpy()[0])
            mse_losses_flag[word]=(mse_loss_start+mse_loss_end)/2
            if(flag=='n' or flag=='v'):
                mse_losses_flag[word]*=1.1
        b = sorted(mse_losses_flag.items(), key=lambda x: x[1],reverse = True)
        word_loss.append(b)
    pbar.close()
    save_word_loss(word_loss, WORD_LOSS_FILE)


### 生成用于评估的数据
评估文件格式要求是3列数据：编号\t预测答案\t证据，我们提供了脚本将模型输出结果转成评估所需格式。

In [13]:
# import math

# 载入未进行文本过滤的测试数据，用于计算TopK
original_data_path = WORK_ROOT + 'data/test.txt'
original_data_ds, _ = load_test_data(original_data_path)
# Re-sort the token index according to their importance scores
def resort(index_array, importance_score):
    res = sorted([[idx, importance_score[idx]] for idx in index_array], key=lambda x:x[1], reverse=True)
    res = [n[0] for n in res]
    return res

def word_vote(original_data_ds, data_ds, idx, context_sent_pos, importance_score, loss, topk, weight, L, rationale_scale):
    assert sum(weight) == 1.0

    importance_score -= np.min(importance_score)
    assert all(importance_score >= 0.0), importance_score

    eval_data_rationale = np.argpartition(importance_score, -topk)[-topk:]
    rationale_index_loss = [data_ds[idx]['context'].find(item[0]) for item in loss]
    rationale_index_score = list(eval_data_rationale)

    indexes = copy.deepcopy(rationale_index_score)
    s = '，；：、,;:（）()【】[]{}<>《》“”""‘’#$%^&*`.¥%……&*〉'+"''"
    for index in indexes:
        word = data_ds[idx]['sent_token'][index]
        if word in s:
            rationale_index_score.remove(index)
    
    sent_num = len(context_sent_pos[idx]) - 1
    vote = np.zeros(sent_num)
    score_sum = np.sum(importance_score[rationale_index_score])
    j = 0
    for pos in rationale_index_score:
        for i in range(sent_num):
            if context_sent_pos[idx][i]<=pos and context_sent_pos[idx][i+1]>pos:
                j += 1
                vote[i] += (importance_score[pos] / score_sum) * weight[0]
    
    loss_list = [item[1] for item in loss]
    loss_sum = sum(loss_list)
    for index, pos in enumerate(rationale_index_loss):
        for i in range(sent_num):
            if context_sent_pos[idx][i]<=pos and context_sent_pos[idx][i+1]>pos:
                vote[i] += loss_list[index] / loss_sum * weight[1]

    sent_index_min_max = np.argsort(vote)
    sent_index_max_min = sent_index_min_max[::-1]
    '''
    if idx == 613:
        print(j)
        print(score_sum)
        print(rationale_index_score)
        print(importance_score[rationale_index_score])
        print(vote)
        print(idx, np.sum(vote), sent_index_max_min)
    '''
    # assert np.sum(vote) > 0.999, np.sum(vote)

    eval_data_rationale = []
    v = 0
    for sent_index in sent_index_max_min:
        v += vote[sent_index]
        sent = "".join([data_ds[idx]['sent_token'][word_index] for word_index in [i for i in range(context_sent_pos[idx][sent_index], context_sent_pos[idx][sent_index+1]-1)]])
        original_sent_pos = original_data_ds[idx]['context'].find(sent[:-1])
        eval_data_rationale.extend([i for i in range(original_sent_pos, original_sent_pos+len(sent))])
        if len(eval_data_rationale) >= rationale_scale*topk and v>=L:  # ratio范围
            break

    return eval_data_rationale

# 查看中文证据(id, answer, question, chinese_rationales)
def get_detail_rationales(interpretation_file, rationale_file, output):
    
    rationales_fomat = []
    with open(rationale_file, 'r') as f:
        for line in f.readlines():
            id, answer, rationale_id = line.split('\t')
            rationale_id = [int(id) for id in rationale_id.split(',')]
            rationales_fomat.append([id, answer, rationale_id])
            
    data = load_data(interpretation_file)
    detail_rationales = ['id\tanswer\tquestion\trationales\n\n']
    for rationale in rationales_fomat:
        id = int(rationale[0])
        sent_token = data[id]['sent_token']
        answer = rationale[1]
        question = data[id]['question']
        rationale_id = rationale[2]
        # print(id, rationale_id)
        rationale_words = "".join([sent_token[id] for id in rationale_id])
        detail_rationales.append("\t".join([str(id), answer, question, rationale_words]) + '\n')
    
    with open(output, 'w') as f:
        f.writelines(detail_rationales)


# Post-prepare the result data so that it can be used for the evaluation directly
def prepare_eval_data(data, results, paddle_model, method, weight, L, rationale_scale):

    res = {}
    idx = 0
    for data_id, inter_res in zip(data, results):
        
        # Split importance score vectors for query and title from inter_res.word_attributions
        importance_score = np.array(inter_res.word_attributions[1:-1])
        loss = word_loss[idx]
        # Extract topK importance scores，过滤后的文本长度可能已经小于TopK
        topk = min(math.ceil(len(original_data_ds[idx]['sent_token'])*RATIONALE_RATIO), len(importance_score))
        
        eval_data = {}        
        eval_data['id'] = data_id
        label = list(inter_res.pred_label)
        if int(label[0])>=int(label[1])+1:
            eval_data['pred_label'] = 'None'
        else:
            eval_data['pred_label'] = ''.join(tokenizer.convert_ids_to_tokens(data_ds_new[idx]['input_ids'][int(label[0]):int(label[1])+1]))
        if method == 'word_vote':
            eval_data['rationale'] = word_vote(original_data_ds, data_ds, idx, context_sent_pos, importance_score, loss, topk, weight, L, rationale_scale)
        else:
            # Find the token index of the topK importance scores
            eval_data['rationale'] = np.argpartition(importance_score, -topk)[-topk:]
            # Re-sort the token index according to their importance scores
            eval_data['rationale'] = resort(eval_data['rationale'], importance_score)

        res[data_id] = eval_data
        idx += 1
    return res

# Generate results for evaluation
att_weight = 0.0
L = 0.0
rationale_scale = 1.0
method = 'word_vote'
if method == 'word_vote':
    att_weight = float(input('请输入赋予att方法的权重[0,1]'))
    weight = [att_weight, 1-att_weight]
    L = float(input('请输入满足证据输出的句子分数阈值[0,1]'))
    rationale_scale = float(input('请输入证据长度范围[0,1]'))

predicts= prepare_eval_data(data, align_res, model, method, weight, L, rationale_scale)

out_file = open(RESULT_PATH + 'mrc_rationale.txt', 'w')
for key in predicts:
    out_file.write(str(predicts[key]['id'])+'\t'+ str(predicts[key]['pred_label'])+'\t')
    for idx in predicts[key]['rationale'][:-1]:
        out_file.write(str(idx)+',')
    if predicts[key]['rationale'] != []:
        out_file.write(str(predicts[key]['rationale'][-1])+'\n')
    else:
        out_file.write(str(0)+'\n')
out_file.close()

# 将输出以原文本为模版查看具体的中文证据

get_detail_rationales(DATA_PATH+'test.txt', RESULT_PATH+'mrc_rationale.txt', RESULT_PATH+'detail_mrc_rationale.txt')

Num of data: 1855


请输入赋予att方法的权重[0,1] 1
请输入满足证据输出的句子分数阈值[0,1] 0.1
请输入证据长度范围[0,1] 0.8


In [16]:
# 实验超参数记录
hyperparameters = {}
hyperparameters['version'] = WORK_ROOT[-4:-1]
hyperparameters['QA_model'] = MODEL_PATH
if 'further' in DATA_FILE:
    hyperparameters['test_data'] = {'file': DATA_FILE, 'further select threshold': THRESHOLD}
    if 'add' in DATA_FILE:
        hyperparameters['test_data']['max_seq_length'] = max_seq_length
else:
    hyperparameters['test_data'] = DATA_FILE

hyperparameters['interpretation'] = {'method': method, 'weight': weight, 'score threhold': L, 'rationale scale': rationale_scale}
hyperparameters['result path'] = RESULT_PATH
depth = 0

def version_record(hyperparameters, record_file, depth):
    padding = depth * '\t'
    for parameter in hyperparameters:
        if type(hyperparameters[parameter]) == type({}):
            record_file.write(f'{padding}{parameter}: \n')
            version_record(hyperparameters[parameter], record_file, depth+1)
        else:
            record_file.write(f'{padding}{parameter}: {hyperparameters[parameter]}\n')

record_file = open(WORK_ROOT + 'version-record.txt', 'w')
version_record(hyperparameters, record_file, depth)

In [5]:
# visual answers
import json
PUBLIC_DATA_PATH = 'work/public_data/'
def generate_refined_dataset(data):
    ref_dataset = json.load(open(PUBLIC_DATA_PATH+'refine_dataset.json'))
    for examples in ref_dataset:
        for example in examples:
            data_id = example['id']
            example['context'] = data[data_id]['context']
            example['sent_token'] = data[data_id]['sent_token']
    
    with open(DATA_PATH + 'refine_dataset.json','w') as f:
        json.dump(ref_dataset,f,indent=4,ensure_ascii=False)


refined_dataset = json.load(open(DATA_PATH+'refine_dataset.json'))  # 待修改数据
ref_dataset = json.load(open(PUBLIC_DATA_PATH+'refine_dataset.json'))

# 可对比查看证据的中文证据转换
def insert2refined_dataset(line:str):
    data = line.rstrip().split("\t")
    id = data[0]
    ans = data[1]
    rationale = [int(v) for v in data[2].split(',')]

    for i, examples in enumerate(refined_dataset):
        for j, example in enumerate(examples):
            if str(example['id']) == id:
                example['ans'] = ans
                example['rationale'] = rationale
                example['rationale_text'] = [ref_dataset[i][j]['context'][idx] for idx in rationale]
                return

generate_refined_dataset(data)

pbar = tqdm(total=1855, ncols=100, desc='已操作文档数目：')
for line in open(RESULT_PATH + 'mrc_rationale.txt','r').readlines():
    pbar.update(1)
    insert2refined_dataset(line)
pbar.close()


with open(RESULT_PATH+'analysis.json','w') as f:
    json.dump(refined_dataset,f,indent=4,ensure_ascii=False)

已操作文档数目：: 100%|███████████████████████████████████████| 1855/1855 [00:00<00:00, 3318.54it/s]


In [49]:
original_data_path = WORK_ROOT + 'data/test.txt'
original_data_ds, data = load_test_data(original_data_path)
out_file = open(RESULT_PATH + 'final_result.txt', 'w')
for line in open(RESULT_PATH + 'mrc_rationale.txt','r').readlines():
    elemts = line.split('\t')
    elemts.insert(2, data[int(elemts[0])]['context'].find(elemts[1]))
    out_file.write(f'{elemts[0]}\t{elemts[1]}\t{elemts[2]}\t{elemts[3]}')
out_file.close()

refined_dataset = json.load(open(RESULT_PATH+'analysis.json'))
for examples in refined_dataset:
    vote = {}
    for example in examples:
        if example['ans'] not in vote and example['ans'] not in 'None[SEP]':
            vote[example['ans']] = 0
        if example['ans'] in vote:
            vote[example['ans']] += 1
    if vote != {}:
        k = max(vote, key=vote.get)  # 找投票最多的答案
    else:
        k = ''
        vote[k] = 1        
    if vote[k] > 1:  # 存在
        commom_rationale = []
        for example in examples:  # 找出该答案的一种证据作为共同证据
            if example['ans'] == k:
                commom_rationale = example['rationale_text']
                break
        for example in examples:  # 共享同一答案和证据
            example['ans'] = k
            rationale_idx = []
            rationale_words = []
            for rationale in commom_rationale:
                if data[int(example['id'])]['context'].find(rationale) != -1:
                    idx = data[int(example['id'])]['context'].find(rationale)
                    rationale_idx.append(idx)
                    rationale_words.append(rationale)
            example['rationale'] = rationale_idx
            example['rationale_text'] = rationale_words
    else:  # 势均力敌的答案，短答案，短证据优先
        a_vote = {}
        r_vote = {}
        for example in examples:
            if example['ans'] not in 'None[SEP]':
                a_vote[example['ans']] = len(example['ans'])
            r_vote["".join(example['rationale_text'])] = len(example['rationale_text'])
        if a_vote != {}:
            ak = min(a_vote, key=a_vote.get)
        else:
            ak = ''
        rk = min(r_vote, key=r_vote.get)
        for example in examples:
            example['ans'] = ak
            rationale_idx = []
            rationale_words = []
            for rationale in rk:
                if data[int(example['id'])]['context'].find(rationale) != -1:
                    idx = data[int(example['id'])]['context'].find(rationale)
                    rationale_idx.append(idx)
                    rationale_words.append(rationale)
            example['rationale'] = rationale_idx
            example['rationale_text'] = rationale_words            


with open(RESULT_PATH+'analysis_union.json','w') as f:
    json.dump(refined_dataset,f,indent=4,ensure_ascii=False)

Num of data: 1855


In [50]:
refined_dataset = json.load(open(RESULT_PATH+'analysis_union.json'))
out_file = open(RESULT_PATH + 'mrc_rationale_union.txt', 'w')

for line in open(RESULT_PATH + 'mrc_rationale.txt','r').readlines():
    elemts = line.split('\t')
    for examples in refined_dataset:
        for example in examples:
            if str(example['id']) == elemts[0]:
                out_file.write(f'{elemts[0]}\t{example["ans"]}\t')
                for idx in example['rationale'][:-1]:
                    out_file.write(str(idx)+',')
                if example['rationale'] != []:
                    out_file.write(str(example['rationale'][-1])+'\n')
                else:
                    out_file.write(str(0)+'\n')
out_file.close()

In [51]:
# 查看中文证据(id, answer, question, chinese_rationales)
def get_detail_rationales(interpretation_file, rationale_file, output):
    
    rationales_fomat = []
    with open(rationale_file, 'r') as f:
        for line in f.readlines():
            id, answer, rationale_id = line.split('\t')
            if rationale_id != '\n':
                rationale_id = [int(id) for id in rationale_id.split(',')]
            else:
                rationale_id = [0]
            rationales_fomat.append([id, answer, rationale_id])
            
    data = load_data(interpretation_file)
    detail_rationales = ['id\tanswer\tquestion\trationales\n\n']
    for rationale in rationales_fomat:
        id = int(rationale[0])
        sent_token = data[id]['sent_token']
        answer = rationale[1]
        question = data[id]['question']
        rationale_id = rationale[2]
        # print(id, rationale_id)
        rationale_words = "".join([sent_token[id] for id in rationale_id])
        detail_rationales.append("\t".join([str(id), answer, question, rationale_words]) + '\n')
    
    with open(output, 'w') as f:
        f.writelines(detail_rationales)
get_detail_rationales(DATA_PATH+'test.txt', RESULT_PATH+'mrc_rationale_union.txt', RESULT_PATH+'detail_mrc_rationale_union.txt')
